In [1]:
import pandas as pd
import numpy as np
import re
#from model_adapter import FeatureExtractor

In [2]:
df_path = "data/df_raw.pkl"
df = pd.read_pickle(df_path).reset_index(drop=True)
df.task_type.value_counts()

Обслуживание    48559
Инцидент        23788
Изменение        9427
Продажа          2510
Name: task_type, dtype: int64

In [3]:
len(df)

84284

In [8]:
df.drop(["uid","task_id"],axis=1).head()

,subject,description,date,org,task_type,year
0,комп 947,не работает скайп комп 947\n\nПользователь Кир...,4013-03-06 17:33:49,Менеджеры техподдержки,Изменение,2013
1,Массовая рассылка(Quick Sales),Рассмотерть возможность массовой рассылки чере...,4013-03-06 17:47:23,Менеджеры техподдержки,Изменение,2013
2,Корпорация СКИП,Завести клиента ЗАО «Корпорация СКИП» \n109428...,4013-03-07 09:56:01,SD,Обслуживание,2013
3,tserv,Тормозит tserv\n\nЛожкин Кирилл Александрович\...,4013-03-07 10:09:11,Инженеры техподдержки,Изменение,2013
4,Outlook,При ответе на письмо и вставки таблицы из Exce...,4013-03-07 11:19:29,Менеджеры техподдержки,Изменение,2013


In [29]:
df["description"].sample(3).iloc[0]

'Проблема с базой АВИ БП\nДобрый день.\r\n \r\nВ базе 1С АВИ БП слетел начисленный аванс по налогу по УСН за 1 квартал 2019\r\nг. - вместо 19 тысяч, которые и были уплачены, теперь показывает 37 тысяч.\r\nТакое могло случиться из-за изменений, внесенных задним числом в 1 квартал в\r\nУНФ и при синхронизации их с БП. Хотя дата запрета изменений в АВИ БП всегда\r\nстояла 31.03.2019.\r\nПросьба разобраться в ситуации и вернуть прежнее начисление.\r\n \r\n \r\n \r\nС Уважением,\r\n \r\n <http://www.aviconsult.ru/> cid:image001.png@01D21276.9FB3D750\r\nКапштык Святослав Евгеньевич\r\nБухгалтер\r\nООО <АВИ Консалт>'

In [30]:
df["split_description"] = df['description'].apply(lambda x: set(re.split('\n| ', x)))
df["split_description"].head(3)

0    {, Телефон, не, (Рабочий, 947, (Мобильный, \t7...
1    {поздравления, , Телефон, (Рабочий, массовой, ...
2    {, www.zao-skip.ru, Телефон, Рязанский, Сергей...
Name: split_description, dtype: object

In [31]:
#dfcopy=df.copy()
def subset (sub : set):
    return {'Инициатор','Клиент:','Направление:','Приоритет:'}

In [35]:
fn_list = [str.lower, str.upper]

def pipeline(text, fn_list):
    # fn_list = [str.to_lower, str.upper]
    for fn in fn_list:
        text = fn(text)
    return text

df['description'].iloc[:10].apply(pipeline, fn_list=fn_list)

0    НЕ РАБОТАЕТ СКАЙП КОМП 947\n\nПОЛЬЗОВАТЕЛЬ КИР...
1    РАССМОТЕРТЬ ВОЗМОЖНОСТЬ МАССОВОЙ РАССЫЛКИ ЧЕРЕ...
2    ЗАВЕСТИ КЛИЕНТА ЗАО «КОРПОРАЦИЯ СКИП» \n109428...
3    ТОРМОЗИТ TSERV\n\nЛОЖКИН КИРИЛЛ АЛЕКСАНДРОВИЧ\...
4    ПРИ ОТВЕТЕ НА ПИСЬМО И ВСТАВКИ ТАБЛИЦЫ ИЗ EXCE...
5    ПРИ ПОПЫТКЕ ОТКРЫТЬ ПРИКРЕПЛЕННЫЙ ФАЙЛ ВЫДАЕТС...
6    ПРОШУ ДАТЬ ДОСТУП ЧТЕНИИ/ЗАПИСЬ ПОЛЬЗОВАТЕЛЮ V...
7    НЕ РАБОТАЕТ АМАДЕУС.\n\nПЛАТОНОВА ЕЛЕНА\n\nE-M...
8    В ДАННЫЙ МОМЕНТ ДОСТУП НАСТРОЕН 194.67.159.246...
9    НА ПОЧТЕ В GMAIL ПРИ НАБОРЕ ТЕКСТА ПРОСТАВЛЯЮТ...
Name: description, dtype: object

In [32]:
def subset(sub : set):
    return {'Инициатор:','Клиент:','Направление:','Приоритет:'}.issubset(sub)

In [33]:
df["bool_split_description"] = df["split_description"].apply(subset)

In [39]:
df["bool_split_description"].head(3)

0    False
1    False
2    False
Name: bool_split_description, dtype: bool

In [34]:
df.groupby('bool_split_description').count()

,subject,description,task_id,date,org,uid,task_type,year,split_description
bool_split_description,,,,,,,,,
False,83845,83845,83845,83845,83845,83845,83845,83845,83845
True,439,439,439,439,439,439,439,439,439


In [35]:
len(df[df.bool_split_description == True])

439

In [36]:
df=df[df.bool_split_description == False]
len(df)

83845

In [37]:
df.task_type.unique()

array(['Изменение', 'Обслуживание', 'Инцидент', 'Продажа'], dtype=object)

In [38]:
df["description_merge"]=df.apply(lambda row: FeatureExtractor.merge_text(row['subject'], row['description']), axis=1)

In [39]:
 df.to_pickle('data/df_first.pkl')